# Tournament Notebook

In [ ]:
## Setup and Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import time
import json
import importlib
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock
from typing import Dict, List, Tuple, Any
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from IPython.display import display, clear_output
import warnings
warnings.filterwarnings('ignore')

# Add src directory to Python path
import sys
sys.path.append(os.path.join('..', 'src'))

# Visualization setup
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 8)

In [ ]:
# Tournament configuration
CONFIG = {
    'max_rounds': 300,
    'max_parallel_workers': 4,  # Set to 1 to disable multithreading for debugging
    'evaluation_metric': 'f1_score',  # 'accuracy', 'f1_score', 'precision', 'recall'
    'cross_validation_folds': 3,
    'sample_size': None,
    'preprocessing_report': False,
    'debug_mode': False,  # Enable detailed logging and disable threading
    
    # Multi-seed configuration
    'num_seeds': 5,  # Number of different random seeds to test
    'seeds': None,   # Will be auto-generated if None
    'parallel_seeds': True,  # Run seeds in parallel
    'seed_aggregation': 'mean',  # How to aggregate across seeds: 'mean', 'median'
}

# Available datasets
AVAILABLE_DATASETS = {
    'bank': os.path.join('..', 'data', 'real', 'bank.csv'),
    'credit': os.path.join('..', 'data', 'real', 'credit.csv'),
    'income': os.path.join('..', 'data', 'real', 'income.csv'),
    'train_A': os.path.join('..', 'data', 'synthetic', 'train_A.csv'),
    'train_B': os.path.join('..', 'data', 'synthetic', 'train_B.csv'),
    'train_C': os.path.join('..', 'data', 'synthetic', 'train_C.csv')
}

In [ ]:
## Dynamic Model and Optimizer Loading
def load_models():
    """Automatically load all models from the models directory (including subdirectories)"""
    models = {}
    models_dir = os.path.join('..', 'src', 'models')
    
    for root, dirs, files in os.walk(models_dir):
        for file in files:
            if file.endswith('.py') and not file.startswith('__') and file != 'BaseModel.py':
                # Get relative path from models directory
                rel_path = os.path.relpath(root, models_dir)
                module_name = file[:-3]  # Remove .py extension
                
                # Build module import path
                if rel_path == '.':
                    import_path = f'models.{module_name}'
                else:
                    import_path = f'models.{rel_path.replace(os.sep, ".")}.{module_name}'
                
                try:
                    module = importlib.import_module(import_path)
                    
                    # Look for classes that are actually defined in this module
                    found_class = None
                    for attr_name in dir(module):
                        attr = getattr(module, attr_name)
                        if (isinstance(attr, type) and 
                            not attr_name.startswith('_') and
                            attr_name != 'BaseModel' and
                            # Check if class is defined in this module
                            attr.__module__ == module.__name__):
                            
                            # Check if it's likely a model class
                            if (attr_name.endswith('Model') or 
                                attr_name == module_name or
                                'Model' in attr_name):
                                found_class = attr
                                break
                    
                    if found_class:
                        clean_name = module_name.replace('Model', '') if module_name.endswith('Model') else module_name
                        # Add subfolder prefix if in subdirectory
                        if rel_path != '.':
                            clean_name = f"{rel_path.replace(os.sep, '_')}_{clean_name}"
                        models[clean_name] = found_class
                        print(f"Loaded model: {clean_name} (class: {found_class.__name__})")
                    else:
                        print(f"No model class found in {import_path}")
                        
                except Exception as e:
                    print(f"Failed to load model {import_path}: {e}")
    
    return models

def load_optimizers():
    """Automatically load all optimizers from the optimizers directory (including subdirectories)"""
    optimizers = {}
    optimizers_dir = os.path.join('..', 'src', 'optimizers')
    
    for root, dirs, files in os.walk(optimizers_dir):
        for file in files:
            if file.endswith('.py') and not file.startswith('__') and file != 'BaseOptimizer.py':
                # Get relative path from optimizers directory
                rel_path = os.path.relpath(root, optimizers_dir)
                module_name = file[:-3]  # Remove .py extension
                
                # Build module import path
                if rel_path == '.':
                    import_path = f'optimizers.{module_name}'
                else:
                    import_path = f'optimizers.{rel_path.replace(os.sep, ".")}.{module_name}'
                
                try:
                    module = importlib.import_module(import_path)
                    
                    # Look for classes that are actually defined in this module
                    found_class = None
                    for attr_name in dir(module):
                        attr = getattr(module, attr_name)
                        if (isinstance(attr, type) and 
                            not attr_name.startswith('_') and
                            attr_name != 'BaseOptimizer' and
                            # Check if class is defined in this module
                            attr.__module__ == module.__name__):
                            
                            # Check if it's likely an optimizer class
                            if (attr_name.endswith('Optimizer') or 
                                attr_name == module_name or
                                'Optimizer' in attr_name):
                                found_class = attr
                                break
                    
                    if found_class:
                        clean_name = module_name.replace('Optimizer', '') if module_name.endswith('Optimizer') else module_name
                        # Add subfolder prefix if in subdirectory
                        if rel_path != '.':
                            clean_name = f"{rel_path.replace(os.sep, '_')}_{clean_name}"
                        optimizers[clean_name] = found_class
                        print(f"Loaded optimizer: {clean_name} (class: {found_class.__name__})")
                    else:
                        print(f"No optimizer class found in {import_path}")
                        
                except Exception as e:
                    print(f"Failed to load optimizer {import_path}: {e}")
    
    return optimizers

print("Loading models...")
AVAILABLE_MODELS = load_models()

print("Loading optimizers...")
AVAILABLE_OPTIMIZERS = load_optimizers()

print(f"Loaded {len(AVAILABLE_MODELS)} models and {len(AVAILABLE_OPTIMIZERS)} optimizers")

In [ ]:
## Dataset Loading and Preprocessing

from data.preprocessing import preprocess_dataset
from utils import TournamentResults, MultiSeedTournamentResults

def load_and_preprocess_dataset(dataset_name: str, sample_size: int = None):
    """Load dataset with optional sampling for speed"""
    X, y, features = preprocess_dataset(AVAILABLE_DATASETS[dataset_name], verbose=CONFIG["preprocessing_report"])
    
    # Automatische Label-Konvertierung für alle Modelle
    if hasattr(y, 'dtype') and y.dtype == 'object':  # String labels
        le = LabelEncoder()
        y = le.fit_transform(y)
        print(f"Converted string labels to numeric: {le.classes_}")
    
    if sample_size and len(X) > sample_size:
        indices = np.random.choice(len(X), sample_size, replace=False)
        X = X[indices]
        y = y[indices]
        print(f"Sampled {sample_size} rows from {len(X)} total")
    
    return X, y, features

if CONFIG['seeds'] is None:
    np.random.seed(42)  # Für reproduzierbare Seed-Generierung
    CONFIG['seeds'] = np.random.randint(1, 10000, CONFIG['num_seeds']).tolist()

print(f"Using seeds: {CONFIG['seeds']}")

# tournament_results = TournamentResults()
tournament_results = MultiSeedTournamentResults(CONFIG['seeds'])

In [ ]:
# Select dataset (available: bank, credit, income, train_A, train_B, train_C)
SELECTED_DATASET = 'train_B'

# Select model (set to None to use first available)
SELECTED_MODEL = "EnhancedMLP"  # e.g. 'RandomForest', 'MLP', 'SVM'

# Select optimizers (set to None to use all available)
SELECTED_OPTIMIZERS = ['rl_OptunaHybrid', 'rl_BayesianRL', 'rl_DQN', 'rl_EpsilonGreedy', 'rl_GradientBandit', 'rl_HybridBandit', 'rl_IndependentBandit', 'rl_OptunaHybrid', 'rl_OptunaTPE', 'rl_PPO']
#["ea_ParticleSwarm", "ea_MuCommaLambdaES", "ea_CMAES"]  e.g. ['rl_OptunaHybrid', 'RandomSearch', 'GridSearch'] or ['rl_DQN', 'rl_GradientBandit' 'ea_GeneticAlgorithm', 'rl_BayesianRL', 'ea_DifferentialEvolution', 'ea_EvolutionStrategyOptimizeer']


In [ ]:
## Tournament Execution Engine

def validate_tournament_configuration():
    """Validate and finalize tournament configuration"""
    
    print("\nTournament Configuration")
    print("=" * 40)
    
    # Show available options
    print(f"Available datasets: {list(AVAILABLE_DATASETS.keys())}")
    print(f"Available models: {list(AVAILABLE_MODELS.keys())}")
    print(f"Available optimizers: {list(AVAILABLE_OPTIMIZERS.keys())}")
    
    # Validate dataset
    dataset = SELECTED_DATASET
    if dataset not in AVAILABLE_DATASETS:
        print(f"Dataset '{dataset}' not found, using first available")
        dataset = list(AVAILABLE_DATASETS.keys())[0]
    
    # Validate model
    model = SELECTED_MODEL
    if model is None or model not in AVAILABLE_MODELS:
        if model is not None:
            print(f"Model '{model}' not found, using first available")
        model = list(AVAILABLE_MODELS.keys())[0]
    
    # Validate optimizers
    optimizers = SELECTED_OPTIMIZERS
    if optimizers is None:
        optimizers = list(AVAILABLE_OPTIMIZERS.keys())
    else:
        valid_optimizers = []
        for opt in optimizers:
            if opt in AVAILABLE_OPTIMIZERS:
                valid_optimizers.append(opt)
            else:
                print(f"Optimizer '{opt}' not found, skipping")
        optimizers = valid_optimizers if valid_optimizers else list(AVAILABLE_OPTIMIZERS.keys())
    
    print(f"\nFinal tournament configuration:")
    print(f"  • Dataset: {dataset}")
    print(f"  • Model: {model}")
    print(f"  • Optimizers: {optimizers}")
    
    return dataset, model, optimizers

# Validate configuration
selected_dataset, selected_model, selected_optimizers = validate_tournament_configuration()

def evaluate_model_performance(model, X, y, hyperparameters: Dict, seed: int = None) -> float:
    """Evaluate model performance using cross-validation with optional seed"""
    # Create and configure model
    model.create_model(hyperparameters)
    
    # Use seed if provided, otherwise use default
    random_state = seed if seed is not None else 42
    cv = StratifiedKFold(n_splits=CONFIG['cross_validation_folds'], shuffle=True, random_state=random_state)
    
    # Choose scoring metric
    scoring_map = {
        'accuracy': 'accuracy',
        'f1_score': 'f1_macro',
        'precision': 'precision_macro',
        'recall': 'recall_macro'
    }
    scoring = scoring_map.get(CONFIG['evaluation_metric'], 'f1_macro')
    
    # Perform cross-validation
    try:
        if CONFIG.get('debug_mode', False):
            scores = cross_val_score(model.model, X, y, cv=cv, scoring=scoring, n_jobs=1)
            if seed is not None:
                print(f"  CV scores (seed {seed}): {scores}")
            print(f"  Mean CV score: {np.mean(scores):.4f}")
        else:
            scores = cross_val_score(model.model, X, y, cv=cv, scoring=scoring, n_jobs=-1)
        return float(np.mean(scores))
    except Exception as e:
        print(f"Error in evaluation: {e}")
        return 0.0

def run_single_tournament_round(optimizer_name: str, round_num: int, X, y, 
                               model_class, optimizer_instance, seed: int = None) -> Dict:
    """Run a single round for one optimizer, optionally with specific seed"""
    start_time = time.time()
    
    try:
        # Create fresh model instance
        model_instance = model_class()
        
        # Set seed for this specific run if provided
        if seed is not None:
            np.random.seed(seed + round_num + hash(optimizer_name) % 1000)
        
        # Get hyperparameter suggestion
        suggested_params = optimizer_instance.suggest_hyperparameters(round_num)
        
        # Evaluate model performance
        score = evaluate_model_performance(model_instance, X, y, suggested_params, seed)
        
        # Update optimizer with results
        optimizer_instance.update(suggested_params, score)
        
        training_time = time.time() - start_time
        
        return {
            'success': True,
            'seed': seed,
            'optimizer': optimizer_name,
            'round': round_num,
            'score': score,
            'params': suggested_params,
            'time': training_time
        }
        
    except Exception as e:
        return {
            'success': False,
            'seed': seed,
            'optimizer': optimizer_name,
            'round': round_num,
            'error': str(e),
            'time': time.time() - start_time
        }
    
def run_tournament_round(round_num: int, X, y, model_class, 
                        optimizer_instances: Dict, 
                        tournament_results: MultiSeedTournamentResults):
    """Run a complete round across all seeds and optimizers"""
    
    round_start_time = time.time()
    
    if CONFIG['parallel_seeds'] and not CONFIG.get('debug_mode', False):
        # Parallel execution across seeds and optimizers
        max_workers = min(CONFIG['max_parallel_workers'], len(CONFIG['seeds']) * len(optimizer_instances))
        
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = []
            
            # Submit tasks for each seed-optimizer combination
            for seed in CONFIG['seeds']:
                for optimizer_name, optimizer_instance in optimizer_instances.items():
                    future = executor.submit(
                        run_single_tournament_round,
                        optimizer_name, round_num, X, y, 
                        model_class, optimizer_instance, seed
                    )
                    futures.append(future)
            
            # Process completed tasks
            for future in as_completed(futures):
                result = future.result()
                if result['success']:
                    tournament_results.add_result(
                        result['seed'], result['optimizer'], result['round'],
                        result['params'], result['score'], result['time']
                    )
                else:
                    print(f"Error in seed {result['seed']}, optimizer {result['optimizer']}: {result.get('error', 'Unknown error')}")
    
    else:
        # Sequential execution for debugging
        for seed in CONFIG['seeds']:
            for optimizer_name, optimizer_instance in optimizer_instances.items():
                if CONFIG.get('debug_mode', False):
                    print(f"Running seed {seed}, optimizer {optimizer_name}")
                
                result = run_single_tournament_round(
                    optimizer_name, round_num, X, y, 
                    model_class, optimizer_instance, seed
                )
                
                if result['success']:
                    tournament_results.add_result(
                        result['seed'], result['optimizer'], result['round'],
                        result['params'], result['score'], result['time']
                    )
                else:
                    print(f"Error: {result.get('error', 'Unknown error')}")
    
    round_time = time.time() - round_start_time
    return round_time

## Live Visualization Functions

def create_live_tournament_dashboard(show_text=False):
    """Create enhanced live dashboard with error bars and seed variance"""
    
    if not CONFIG["debug_mode"]:
        clear_output(wait=True)
    
    standings = tournament_results.get_current_standings_with_variance()
    
    if standings.empty:
        print("No results yet...")
        return
    
    # Create figure with subplots (ohne große Tabelle)
    fig = plt.figure(figsize=(20, 10))
    gs = fig.add_gridspec(1, 3, hspace=0.3, wspace=0.5)  # Nur eine Reihe
    
    # Main title
    current_round = max([max(data['rounds']) if data['rounds'] else 0 
                        for seed_data in tournament_results.seed_results.values() 
                        for data in seed_data.values()], default=0)
    
    fig.suptitle(f'Live Multi-Seed Tournament Dashboard - Round {current_round}', 
                fontsize=16, fontweight='bold')
    
    # 1. Performance comparison with error bars (verbessert für viele Optimizer)
    ax1 = fig.add_subplot(gs[0, 0])
    optimizers = standings['Optimizer'].values
    means = standings['Mean Best Score'].values
    stds = standings['Std Best Score'].values
    
    colors = plt.cm.Set1(np.linspace(0, 1, len(optimizers)))
    
    # Bessere Y-Achsen Skalierung
    y_min = max(0, min(means) - max(stds) - 0.02)
    y_max = max(means) + max(stds) + 0.05  # Mehr Platz für Labels
    
    # Vergrößerte Fehlerbalken und bessere Sichtbarkeit
    bars = ax1.bar(range(len(optimizers)), means, 
                   color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
    
    # Separate Fehlerbalken für bessere Sichtbarkeit
    ax1.errorbar(range(len(optimizers)), means, yerr=stds, 
                fmt='none', color='black', capsize=6, capthick=1.5, linewidth=1.5)
    
    ax1.set_xlabel('Optimizers', fontsize=12)
    ax1.set_ylabel('Best Score (Mean ± Std)', fontsize=12)
    ax1.set_title('Performance Comparison', fontsize=14, fontweight='bold', pad=20)
    ax1.set_xticks(range(len(optimizers)))
    
    # Bessere Label-Rotation basierend auf Anzahl
    if len(optimizers) <= 4:
        ax1.set_xticklabels(optimizers, rotation=0, ha='center')
        label_fontsize = 9
    elif len(optimizers) <= 8:
        ax1.set_xticklabels(optimizers, rotation=45, ha='right')
        label_fontsize = 8
    else:
        ax1.set_xticklabels(optimizers, rotation=90, ha='right')
        label_fontsize = 7
    
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(y_min, y_max)
    
    # Intelligente Label-Positionierung um Überlappungen zu vermeiden
    for i, (mean, std) in enumerate(zip(means, stds)):
        cv = (std/mean*100) if mean > 0 else 0
        
        # Alterniere Label-Position bei vielen Optimizern
        if len(optimizers) > 6:
            offset = 0.015 if i % 2 == 0 else 0.025
            label_text = f'{mean:.3f}\n±{std:.3f}'  # Kompakter
        else:
            offset = 0.015
            label_text = f'{mean:.3f}±{std:.3f}\n({cv:.1f}%)'
        
        ax1.text(i, mean + std + offset, label_text, 
                ha='center', va='bottom', fontsize=label_fontsize, fontweight='bold',
                bbox=dict(boxstyle='round,pad=0.2', facecolor='white', alpha=0.9))
    
    # 2. Score progression over rounds (verbessert für mehr Optimizer)
    ax2 = fig.add_subplot(gs[0, 1])
    
    # Zeige alle Optimizer, aber mit intelligenteren Linien
    max_optimizers_to_show = min(len(optimizers), 8)  # Maximal 8 statt 4
    
    for i, optimizer in enumerate(optimizers[:max_optimizers_to_show]):
        score_history = tournament_results.get_score_history_by_seed(optimizer)
        
        # Calculate statistics across seeds for each round
        max_rounds = max(len(scores) for scores in score_history.values() if scores)
        if max_rounds > 0:
            round_means = []
            round_stds = []
            
            for round_idx in range(max_rounds):
                round_scores = []
                for seed_scores in score_history.values():
                    if round_idx < len(seed_scores):
                        round_scores.append(seed_scores[round_idx])
                
                if round_scores:
                    round_means.append(np.mean(round_scores))
                    round_stds.append(np.std(round_scores))
                else:
                    round_means.append(np.nan)
                    round_stds.append(np.nan)
            
            rounds = range(1, len(round_means) + 1)
            round_means = np.array(round_means)
            round_stds = np.array(round_stds)
            
            # Dünnere Linien und verschiedene Stile für bessere Unterscheidung
            line_styles = ['-', '--', '-.', ':']
            line_style = line_styles[i % len(line_styles)]
            line_width = 2.0 if i < 4 else 1.5  # Top 4 dicker
            alpha = 1.0 if i < 4 else 0.8  # Top 4 mehr opacity
            
            # Plot mean line
            ax2.plot(rounds, round_means, label=optimizer, color=colors[i], 
                    linewidth=line_width, linestyle=line_style, alpha=alpha, marker='o', markersize=3)
            
            # Add confidence interval nur für Top 4 um Clutter zu vermeiden
            if i < 4:
                ax2.fill_between(rounds, 
                               round_means - round_stds, 
                               round_means + round_stds, 
                               alpha=0.2, color=colors[i])
    
    ax2.set_xlabel('Round', fontsize=12)
    ax2.set_ylabel('Score', fontsize=12)
    ax2.set_title('Score Progression', fontsize=14, fontweight='bold')
    
    # Bessere Legende basierend auf Anzahl Optimizer
    if len(optimizers) <= 4:
        ax2.legend(loc='upper left', fontsize=10)
    elif len(optimizers) <= 6:
        ax2.legend(loc='upper left', fontsize=9, ncol=2)
    else:
        ax2.legend(loc='upper left', fontsize=8, ncol=2)
    
    ax2.grid(True, alpha=0.3)
    
    # 3. Reproducibility Analysis (kompakter)
    ax3 = fig.add_subplot(gs[0, 2])
    
    # Berechne Coefficient of Variation für jeden Optimizer
    cv_values = []
    optimizer_names_cv = []
    
    for _, row in standings.iterrows():
        if row['Mean Best Score'] > 0:
            cv = (row['Std Best Score'] / row['Mean Best Score']) * 100
            cv_values.append(cv)
            optimizer_names_cv.append(row['Optimizer'])
    
    if cv_values:
        # Sortiere nach CV (niedrigster = reproducible)
        sorted_data = sorted(zip(cv_values, optimizer_names_cv))
        cv_values, optimizer_names_cv = zip(*sorted_data)
        
        colors_cv = ['green' if cv < 5 else 'orange' if cv < 10 else 'red' for cv in cv_values]
        
        bars_cv = ax3.barh(range(len(cv_values)), cv_values, color=colors_cv, alpha=0.7, edgecolor='black')
        
        ax3.set_yticks(range(len(optimizer_names_cv)))
        ax3.set_yticklabels(optimizer_names_cv, fontsize=10)
        ax3.set_xlabel('Coefficient of Variation (%)', fontsize=12)
        ax3.set_title('Reproducibility Analysis', fontsize=14, fontweight='bold')
        ax3.grid(True, alpha=0.3, axis='x')
        
        # Add value labels
        for i, cv in enumerate(cv_values):
            ax3.text(cv + 0.05, i, f'{cv:.1f}%', va='center', fontsize=10, fontweight='bold')
        
        # Kompakte Legende innerhalb des Plots
        legend_elements = [
            plt.Rectangle((0,0),1,1, facecolor='green', alpha=0.7, label='<5%'),
            plt.Rectangle((0,0),1,1, facecolor='orange', alpha=0.7, label='5-10%'),
            plt.Rectangle((0,0),1,1, facecolor='red', alpha=0.7, label='>10%')
        ]
        ax3.legend(handles=legend_elements, loc='lower right', fontsize=9, title='CV')
    
    plt.tight_layout()
    plt.show()
    
    # Kompakte Textausgabe statt große Tabelle
    if show_text:
        print(f"\n{'='*60}")
        print(f"TOURNAMENT SUMMARY - Round {current_round}")
        print(f"{'='*60}")
        print(f"Seeds: {len(CONFIG['seeds'])} | Metric: {CONFIG['evaluation_metric']}")
        
        if not standings.empty:
            print(f"\nTOP 3 PERFORMERS:")
            for i, (_, row) in enumerate(standings.head(3).iterrows()):
                cv = (row['Std Best Score']/row['Mean Best Score']*100) if row['Mean Best Score'] > 0 else 0
                medal = "🥇" if i == 0 else "🥈" if i == 1 else "🥉"
                print(f"  {medal} {row['Optimizer']}: {row['Mean Best Score']:.4f}±{row['Std Best Score']:.4f} (CV: {cv:.1f}%)")
            
            # Zusätzliche Insights
            winner = standings.iloc[0]
            winner_cv = (winner['Std Best Score']/winner['Mean Best Score']*100) if winner['Mean Best Score'] > 0 else 0
            
            print(f"\n📊 INSIGHTS:")
            print(f"   • Winner Reproducibility: {'Excellent' if winner_cv < 5 else 'Good' if winner_cv < 10 else 'Poor'}")
            print(f"   • Total Evaluations: {winner['Total Evaluations']}")
            print(f"   • Seeds per Optimizer: {winner['Seeds Completed']}/{len(CONFIG['seeds'])}")
    
    # Print summary statistics if requested
    if show_text:
        print(f"\n{'='*60}")
        print(f"MULTI-SEED TOURNAMENT STATISTICS - Round {current_round}")
        print(f"{'='*60}")
        print(f"Total Seeds: {len(CONFIG['seeds'])}")
        print(f"Seeds: {CONFIG['seeds']}")
        
        if not standings.empty:
            print(f"\nCURRENT LEADER:")
            leader = standings.iloc[0]
            print(f"  {leader['Optimizer']}")
            print(f"    Mean Best Score: {leader['Mean Best Score']:.4f} ± {leader['Std Best Score']:.4f}")
            print(f"    Seeds Completed: {leader['Seeds Completed']}/{leader['Total Seeds']}")
            
            print(f"\nTOP 3 PERFORMERS:")
            for _, row in standings.head(3).iterrows():
                print(f"  {row['Rank']}. {row['Optimizer']}: {row['Mean Best Score']:.4f} ± {row['Std Best Score']:.4f}")

def print_round_status(round_num, max_rounds, round_time):
    """Print round status with seed information"""
    seed_info = f" ({len(CONFIG['seeds'])} seeds)" if CONFIG['num_seeds'] > 1 else ""
    print(f"Round {round_num}/{max_rounds}{seed_info} - {round_time:.1f}s")

In [ ]:
# Load dataset
print(f"Loading dataset: {selected_dataset}")
X, y, feature_names = load_and_preprocess_dataset(selected_dataset, sample_size=CONFIG["sample_size"])

In [ ]:
## Tournament Execution

print(f"\nStarting Enhanced Tournament!")
print(f"Max rounds: {CONFIG['max_rounds']}")
print(f"Seeds: {CONFIG['num_seeds']} ({CONFIG['seeds']})")
print("=" * 60)

# Initialize model and optimizer instances
model_class = AVAILABLE_MODELS[selected_model]
model_instance = AVAILABLE_MODELS[selected_model]()
optimizer_instances = {}

for optimizer_name in selected_optimizers:
    optimizer_instances[optimizer_name] = AVAILABLE_OPTIMIZERS[optimizer_name](
        model_instance.hyperparameter_space
    )

print(f"\nInitialized {len(optimizer_instances)} optimizers")

# Main tournament loop
for round_num in range(1, CONFIG['max_rounds'] + 1):
    
    # Run round across all seeds
    round_time = run_tournament_round(round_num, X, y, model_class, optimizer_instances, tournament_results)
    
    # Update dashboard every round
    if not CONFIG.get('debug_mode', False):
        create_live_tournament_dashboard(show_text=False)
        print_round_status(round_num, CONFIG['max_rounds'], round_time)
    else:
        print(f"\nRound {round_num} completed in {round_time:.1f}s")
        # Show current standings in debug mode
        standings = tournament_results.get_current_standings_with_variance()
        if not standings.empty:
            print("Current standings:")
            for _, row in standings.head(3).iterrows():
                print(f"  {row['Rank']}. {row['Optimizer']}: {row['Mean Best Score']:.4f} ± {row['Std Best Score']:.4f}")
    
    # Show progress every 10 rounds
    if round_num % 10 == 0:
        standings = tournament_results.get_current_standings_with_variance()
        if not standings.empty:
            leader = standings.iloc[0]
            print(f"Current leader: {leader['Optimizer']} "
                  f"({leader['Mean Best Score']:.4f} ± {leader['Std Best Score']:.4f})")

print(f"\nTournament Complete!")

## Final Results and Analysis

print("FINAL TOURNAMENT RESULTS")
print("=" * 30)

# Show final dashboard with text details
create_live_tournament_dashboard(show_text=True)

# Detailed final standings
final_standings = tournament_results.get_current_standings_with_variance()

if not final_standings.empty:
    print(f"\nTOURNAMENT WINNER:")
    winner = final_standings.iloc[0]
    print(f"  {winner['Optimizer']}")
    print(f"     Mean Best Score: {winner['Mean Best Score']:.4f} ± {winner['Std Best Score']:.4f}")
    print(f"     Seeds Completed: {winner['Seeds Completed']}/{winner['Total Seeds']}")
    print(f"     Total Evaluations: {winner['Total Evaluations']}")
    
    # Reproducibility analysis
    cv = (winner['Std Best Score']/winner['Mean Best Score']*100) if winner['Mean Best Score'] > 0 else 0
    print(f"     Coefficient of Variation: {cv:.2f}%")

    print(f"\nTOP 3 PERFORMERS:")
    for _, row in final_standings.head(3).iterrows():
        cv = (row['Std Best Score']/row['Mean Best Score']*100) if row['Mean Best Score'] > 0 else 0
        print(f"  Rank {row['Rank']}: {row['Optimizer']} - {row['Mean Best Score']:.4f}±{row['Std Best Score']:.4f} (CV: {cv:.1f}%)")

# Best hyperparameters analysis (erweitert für Multi-Seed)
print(f"\nBEST HYPERPARAMETERS ANALYSIS:")
print("-" * 40)

for optimizer in final_standings['Optimizer'].head(3):  # Top 3 only
    # Get best hyperparams across all seeds for this optimizer
    best_score = -np.inf
    best_params = None
    
    for seed_data in tournament_results.seed_results.values():
        if optimizer in seed_data and seed_data[optimizer]['best_hyperparams']:
            if seed_data[optimizer]['best_score'] > best_score:
                best_score = seed_data[optimizer]['best_score']
                best_params = seed_data[optimizer]['best_hyperparams']
    
    if best_params:
        print(f"\n{optimizer}:")
        print(f"  Best Score: {best_score:.4f}")
        print("  Best hyperparameters:")
        for param, value in best_params.items():
            if isinstance(value, float):
                print(f"    {param}: {value:.4f}")
            else:
                print(f"    {param}: {value}")

print(f"\n{'='*60}")
print("TOURNAMENT COMPLETE!")
print(f"{'='*60}")